# Деревья решений решают проблемы
__Суммарное количество баллов: 10__

Вы уже знакомы с классификацией методом KNN. В этом задании предстоит реализовать другой метод классификации - дерево решений. 

Одной из его особенностей является возможность объяснить в человекочитаемой форме, почему мы отнесли объект к определенному классу. Эта особенность позволяет использовать деревья решений для создания систем, которые могут подсказывать специалистам, на что именно стоит обратить внимание при принятии решений.

In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import matplotlib
from typing import Tuple

Во время построения дерева решений нам потребуется определить, какой из предикатов лучше всего разбивает обучающую выборку. Есть два критерия, которые позволяют это сделать: критерий Джини и энтропийный критерий. Первый для подсчета информативности разбиения использует коэффициент Джини, второй - энтропию. Реализуйте подсчет этих коэффициентов, а так же подсчет информативности разбиения. 

#### Описание функций
`gini(x)` считает коэффициент Джини для массива меток

`entropy(x)` считает энтропию для массива меток

`gain(left_y, right_y, criterion)` считает информативность разбиения массива меток на левую `left_y` и правую `right_y` части при помощи `criterion`, который задается функцией (не строкой).

In [ ]:
from math import log
from functools import reduce


def get_p(x):
    def tmp(clazz):
        return len([ t for t in x if t == clazz]) / len(x)
    
    return tmp

def gini(x):
    classes = np.unique(x)

    p = get_p(x)

    return reduce(
        lambda acc, clazz: acc + p(clazz) * (1 - p(clazz)),
        classes,
        0
    )
    
def entropy(x):
    classes = np.unique(x)

    p = get_p(x)

    return -reduce(
        lambda acc, clazz: acc + p(clazz) * log(p(clazz)),
        classes,
        0
    )

def gain(left_y, right_y, criterion):
    return (len(left_y) * criterion(left_y) + len(right_y) * criterion(right_y)) / (len(left_y) + len(right_y))

Деревья решений имеют хорошую интерпретируемость, т.к. позволяют не только предсказать класс, но и объяснить, почему мы предсказали именно его. Например, мы можем его нарисовать. Чтобы сделать это, нам необходимо знать, как оно устроено внутри. Реализуйте классы, которые будут задавать структуру дерева. 

#### DecisionTreeLeaf
Поля:
1. `y` должно содержать класс, который встречается чаще всего среди элементов листа дерева

#### DecisionTreeNode
В данной домашней работе мы ограничемся порядковыми и количественными признаками, поэтому достаточно хранить измерение и значение признака, по которому разбиваем обучающую выборку.

Поля:
1. `split_dim` измерение, по которому разбиваем выборку
2. `split_value` значение, по которому разбираем выборку
3. `left` поддерево, отвечающее за случай `x[split_dim] < split_value`. Может быть `DecisionTreeNode` или `DecisionTreeLeaf`
4. `right` поддерево, отвечающее за случай `x[split_dim] >= split_value`. Может быть `DecisionTreeNode` или `DecisionTreeLeaf`

__Интерфейс классов можно и нужно менять при необходимости__ (например, для вычисления вероятности в следующем задании)

In [ ]:
class DecisionTreeLeaf:
    def __init__(self, labels):
        self.probabilities = {}
        unique_labels = np.unique(labels)
        
        for label in unique_labels:
            self.probabilities[label] = len([ y for y in labels if y == label ]) / len(labels)
        
        self.y = reduce(
            lambda max_label, current_label: current_label if self.probabilities[max_label] < self.probabilities[current_label] else max_label,
            self.probabilities
        )

class DecisionTreeNode:
    def __init__(self, split_dim, split_value, left, right):
        self.split_dim = split_dim
        self.split_value = split_value
        self.left = left
        self.right = right

Теперь перейдем к самому дереву решений. Реализуйте класс `DecisionTreeClassifier`.

#### Описание методов
`fit(X, y)` строит дерево решений по обучающей выборке.

`predict_proba(X)` для каждого элемента из `X` возвращает словарь `dict`, состоящий из пар `(класс, вероятность)`. Вероятности классов в листе можно определить через количество объектов соответствующего класса в листе. 

#### Описание параметров конструктора
`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

#### Описание полей
`root` - корень дерева. Может быть `DecisionTreeNode` или `DecisionTreeLeaf`

In [ ]:
class DecisionTreeClassifier:
    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1):
        self.root = None
        self.__gain = lambda left, right: gain(left, right, gini if criterion == 'gini' else entropy)
        self.__max_depth = max_depth
        self.__min_samples_leaf = min_samples_leaf
    
    def fit(self, X, y):
        self.__data = {} # Словарь: Ключ -- id, значение -- пара (пример, метка)
        
        for x in X:
            label = -1
            
            for i in y:
                if i[0] == x[0]:
                    label = i[1]
                    break
            
            if label != -1:
                self.__data[x[0]] = (x[1:], label)
        
        self.root = self.__get_node_or_leaf(self.__data, 1)
                    
    def __get_node_or_leaf(self, data_subset, depth):
        labels = [ data_subset[id][1] for id in data_subset ]
        
        if len(data_subset.items()) <= self.__min_samples_leaf:
            return DecisionTreeLeaf(labels)
        
        if self.__max_depth is not None and self.__max_depth <= depth: # Дререво достигло максимальной глубины
            return DecisionTreeLeaf(labels)
        
        if len(np.unique(labels)) == 1: # Все объекты принадлежат к одному классу
            return DecisionTreeLeaf(labels)
        

        gain = None # Вместо +inf
        split_feature_idx = 0
        split_treshold = 0
        left_data_subset, right_data_subset = {}, {}

        a = np.empty((len(data_subset), len(next(iter(data_subset.values()))[0])))
        print('shape', a.shape)
        counter = 0
        for id in data_subset:
            a[counter] = data_subset[id][0]
            counter+=1
        print(a.reshape((-1, len(data_subset))))
        for feature_idx, feature in enumerate(next(iter(data_subset.values()))[0]): # Достаём первый элемент из словаря
            pass
        
        for sample_id in data_subset:
            sample_with_label = data_subset[sample_id]
            for feature_id, feature_value in enumerate(sample_with_label[0]):                

                samples_with_labels_left, samples_with_labels_right = {}, {}

                for id in data_subset:
                    if data_subset[id][0][feature_id] < feature_value:
                        samples_with_labels_left[id] = data_subset[id]
                    else:
                        samples_with_labels_right[id] = data_subset[id]

                local_gain = self.__gain(
                    [ data_subset[id][1] for id in samples_with_labels_left ],
                    [ data_subset[id][1] for id in samples_with_labels_right ]
                )
                
                if gain is None or local_gain < gain:
                    gain = local_gain
                    split_feature_idx = feature_id
                    split_treshold = feature_value
                    left_data_subset = samples_with_labels_left.copy()
                    right_data_subset = samples_with_labels_right.copy()
        
        return DecisionTreeNode(
            split_feature_idx,
            split_treshold,
            self.__get_node_or_leaf(left_data_subset, depth + 1),
            self.__get_node_or_leaf(right_data_subset, depth + 1)
        )

    
    def predict_proba(self, X):
        result = np.empty(len(X), dtype=object)

        for sample_idx, sample in enumerate(X):
            current_node = self.root

            while type(current_node) != type(DecisionTreeLeaf([0])): # Аргумент, передоваемый в Leaf -- костыль
                split_feature_idx = current_node.split_dim + 1

                if sample[split_feature_idx] < current_node.split_value:
                    current_node = current_node.left
                else:
                    current_node = current_node.right
            
            result[sample_idx] = current_node.probabilities
        
        return result
    
    def predict(self, X):
        proba = self.predict_proba(X)
        return [max(p.keys(), key=lambda k: p[k]) for p in proba]

Построенное дерево можно нарисовать. Метод `draw_tree` рисует дерево и сохраняет его в указанный файл.

In [ ]:
def tree_depth(tree_root):
    if isinstance(tree_root, DecisionTreeNode):
        return max(tree_depth(tree_root.left), tree_depth(tree_root.right)) + 1
    else:
        return 1

def draw_tree_rec(tree_root, x_left, x_right, y):
    x_center = (x_right - x_left) / 2 + x_left
    if isinstance(tree_root, DecisionTreeNode):
        x_center = (x_right - x_left) / 2 + x_left
        x = draw_tree_rec(tree_root.left, x_left, x_center, y - 1)
        plt.plot((x_center, x), (y - 0.1, y - 0.9), c=(0, 0, 0))
        x = draw_tree_rec(tree_root.right, x_center, x_right, y - 1)
        plt.plot((x_center, x), (y - 0.1, y - 0.9), c=(0, 0, 0))
        plt.text(x_center, y, "x[%i] < %f" % (tree_root.split_dim, tree_root.split_value),
                horizontalalignment='center')
    else:
        plt.text(x_center, y, str(tree_root.y),
                horizontalalignment='center')
    return x_center

def draw_tree(tree, save_path=None):
    td = tree_depth(tree.root)
    plt.figure(figsize=(0.33 * 2 ** td, 2 * td))
    plt.xlim(-1, 1)
    plt.ylim(0.95, td + 0.05)
    plt.axis('off')
    draw_tree_rec(tree.root, -1, 1, td)
    plt.tight_layout()
    if save_path is not None:
        plt.savefig(save_path)
    plt.show()

Для двумерного набора данных дерево можно отобразить на плоскости с данными. Кроме того, как и для любого классификатора, для него можно построить roc-кривую

In [ ]:
def plot_roc_curve(y_test, p_pred):
    positive_samples = sum(1 for y in y_test if y == 0)
    tpr = []
    fpr = []
    for w in np.arange(-0.01, 1.02, 0.01):
        y_pred = [(0 if p.get(0, 0) > w else 1) for p in p_pred]
        tpr.append(sum(1 for yp, yt in zip(y_pred, y_test) if yp == 0 and yt == 0) / positive_samples)
        fpr.append(sum(1 for yp, yt in zip(y_pred, y_test) if yp == 0 and yt != 0) / (len(y_test) - positive_samples))
    plt.figure(figsize = (7, 7))
    plt.plot(fpr, tpr)
    plt.plot([0, 1], [0, 1], linestyle="--")
    plt.xlabel("False positive rate")
    plt.ylabel("True positive rate")
    plt.xlim(-0.01, 1.01)
    plt.ylim(-0.01, 1.01)
    plt.tight_layout()
    plt.show()

def rectangle_bounds(bounds):
    return ((bounds[0][0], bounds[0][0], bounds[0][1], bounds[0][1]), 
            (bounds[1][0], bounds[1][1], bounds[1][1], bounds[1][0]))

def plot_2d_tree(tree_root, bounds, colors):
    if isinstance(tree_root, DecisionTreeNode):
        if tree_root.split_dim:
            plot_2d_tree(tree_root.left, [bounds[0], [bounds[1][0], tree_root.split_value]], colors)
            plot_2d_tree(tree_root.right, [bounds[0], [tree_root.split_value, bounds[1][1]]], colors)
            plt.plot(bounds[0], (tree_root.split_value, tree_root.split_value), c=(0, 0, 0))
        else:
            plot_2d_tree(tree_root.left, [[bounds[0][0], tree_root.split_value], bounds[1]], colors)
            plot_2d_tree(tree_root.right, [[tree_root.split_value, bounds[0][1]], bounds[1]], colors)
            plt.plot((tree_root.split_value, tree_root.split_value), bounds[1], c=(0, 0, 0))
    else:
        x, y = rectangle_bounds(bounds)
        plt.fill(x, y, c=colors[tree_root.y] + [0.2])

def plot_2d(tree, X, y):
    plt.figure(figsize=(9, 9))
    colors = dict((c, list(np.random.random(3))) for c in np.unique(y))
    bounds = list(zip(np.min(X, axis=0), np.max(X, axis=0)))
    plt.xlim(*bounds[0])
    plt.ylim(*bounds[1])
    plot_2d_tree(tree.root, list(zip(np.min(X, axis=0), np.max(X, axis=0))), colors)
    for c in np.unique(y):
        plt.scatter(X[y==c, 0], X[y==c, 1], c=[colors[c]], label=c)
    plt.legend()
    plt.tight_layout()
    plt.show()


Протестируйте решение на датасете spam.
Для этой задачи используйте данные x_spam_train и y_spam_train:
1. Выполните загрузку и предобработку файлов x_spam_train и y_spam_train.
2. Разбейте x_spam_train и y_spam_train на x_train, y_train, x_test и y_test для оценки точности работы алгоритма.
3. Посчитайте метрики `precision`, `recall`, `accuracy` для модели Decision Tree. Если необходимо, попробуйте разные наборы параметров для получения лучшего результата.
4. Сравните значения метрик с результатами модели kNN из предыдущего задания (можно использовать реализацию из библиотеки `sklearn`).
5. Ответьте на следующие вопросы:
    - Какой нужен препроцессинг данных для моделей?
    - Какая модель делает предсказания лучше?  Предположите, почему.

_Ваш ответ_

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score

from sklearn.neighbors import KNeighborsClassifier

In [ ]:
def train_test_split(X: pd.DataFrame, y: np.array, ratio: float = 0.7) -> Tuple[np.array, np.array, np.array, np.array]:
    # Возвращает X_train, y_train, X_test, y_test
    # X_train и X_test - массив векторов - две части массива X, разделенного в состветсви с коэффициентом ratio
    # y_train и y_test - соответствующие X_train и X_test метки классов
    treshold: int = int(X.shape[0] * ratio)
    return X[:treshold].to_numpy(), y[:treshold], X[treshold:].to_numpy(), y[treshold:]

X, y = pd.read_csv('hw_trees_data/x_spam_train.csv'), pd.read_csv('hw_trees_data/y_spam_train.csv').to_numpy()
X_train, y_train, X_test, y_test = train_test_split(X, y, ratio=0.3)
tree = DecisionTreeClassifier(max_depth=5, min_samples_leaf=30)
tree.fit(X_train, y_train)
plot_roc_curve([ y[1] for y in y_test ], tree.predict_proba(X_test))
draw_tree(tree)

1. Обучите модель на всех данных из x_spam_train и y_spam_train.
2. Сделайте submit своего решения и получите значение f1_score не менее 0.6

In [ ]:
submission = pd.DataFrame(columns = ["Id", "Expected"])
submission["Id"] = test["Id"]
submission["Expected"] = #YOUR CODE
submission.to_csv('submission.csv', index=False)
